### Security Hub
##### Minimal Cost Configuration

In [ ]:
import botoplus.botoplus as botoplus

accounts = botoplus.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    session = botoplus.defaults(account['awsaccount'])
    ec2 = session.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        try:
            ### STANDARDS ###
            securityhub = session.client('securityhub', region_name = region['RegionName'])
            paginator = securityhub.get_paginator('get_enabled_standards')
            pages = paginator.paginate()
            for page in pages:
                for subscriptions in page['StandardsSubscriptions']:
                    securityhub.batch_disable_standards(
                        StandardsSubscriptionArns = [
                            subscriptions['StandardsSubscriptionArn']
                        ]
                    )
            ### SUBSCRIPTIONS ###
            paginator = securityhub.get_paginator('list_enabled_products_for_import')
            pages = paginator.paginate()
            for page in pages:
                for products in page['ProductSubscriptions']:
                    parsed = products.split('/')
                    if parsed[2] != 'access-analyzer' and parsed[2] != 'config' and parsed[2] != 'guardduty' and parsed[2] != 'health' and parsed[2] != 'inspector' and parsed[2] != 'securityhub':
                        securityhub.disable_import_findings_for_product(
                            ProductSubscriptionArn = products
                        )
            print(' - '+region['RegionName'])
        except:
            print(' - '+region['RegionName']+' DENIED')
            pass

##### List Enabled Region Integrations

In [ ]:
import botoplus.botoplus as botoplus

accounts = botoplus.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    session = botoplus.defaults(account['awsaccount'])
    ec2 = session.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        print(' - '+region['RegionName'])
        securityhub = session.client('securityhub', region_name=region['RegionName'])
        paginator = securityhub.get_paginator('list_enabled_products_for_import')
        pages = paginator.paginate()

        for page in pages:
            for products in page['ProductSubscriptions']:
                print(products)

##### Enable Region Integrations

In [ ]:
import botoplus.botoplus as botoplus

accounts = botoplus.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    session = botoplus.defaults(account['awsaccount'])
    ec2 = session.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        print(' - '+region['RegionName'])
        securityhub = session.client('securityhub', region_name=region['RegionName'])
        paginator = securityhub.get_paginator('describe_products')
        pages = paginator.paginate()

        for page in pages:
            for products in page['Products']:
                if products['CompanyName'] == 'AWS':
                    if products['ProductName'] == 'Config' or products['ProductName'] == 'GuardDuty' or products['ProductName'] == 'Health' or products['ProductName'] == 'IAM Access Analyzer' or products['ProductName'] == 'Inspector':
                        try:
                            securityhub.enable_import_findings_for_product(
                                ProductArn = products['ProductArn']
                            )
                        except:
                            pass